## preprocessing 

to make it compatible with how morph-kgc proceses xml's

- can't deal with missing data
- can't process logic 
    - `IsApplicable` is therefore processed here
    - alongside splitting OperationalPoints into OperationalPoints and OperarionalBorderPoints
- can't get to sibling data (in our case the MemberStateCode)


In [ ]:
# Importing the required libraries
import sys
import xml.etree.ElementTree as et

# tree = et.parse('ES-mini.xml')
tree = et.parse('src/ES-mini-x2.xml')
# tree = et.parse('src/ES.xml')
root = tree.getroot()

# get MemberStateCode 
member_state_codes = root.findall('MemberStateCode')
if len(member_state_codes) != 1:
	sys.exit(69)

# add ValidityDateEnd and MemberStateCode as a child
for op in root.iter('OperationalPoint'):
	# ValidityDateStart
	if 'ValidityDateStart' not in op.attrib:
		op.attrib['ValidityDateStart'] = '1942-09-01'
		print('added ValidityDateStart in ', op.findall('UniqueOPID')[0].attrib['Value'])
	# ValidityDateEnd
	if 'ValidityDateEnd' not in op.attrib:
		op.attrib['ValidityDateEnd'] = '2069-04-20'
		# print('added ValidityDateEnd in ', op.findall('UniqueOPID')[0].attrib['Value'])
	# MemberStateCode
	op.append(member_state_codes[0])

	# OPTafTapCode removal
	for tapcode in op.findall('OPTafTapCode'):
		if tapcode.attrib['IsApplicable'] != 'Y':
			# print('removed tapcode in ', op.findall('UniqueOPID')[0].attrib['Value'])
			if tapcode.attrib['IsApplicable'] == 'NYA':
				tapcode.attrib['Value'] = 'NYA'
			else:
				op.remove(tapcode)

	

# removing unused params
for op in root.iter('OperationalPoint'):
	for track in op.iter('OPTrack'):
		for param in track.findall('OPTrackParameter'):
			if param.attrib['IsApplicable'] != 'Y':
				# print('removed param in track ', track.findall('OPTrackIdentification')[0].attrib['Value'])
				track.remove(param)

# removing unused tunnel and platform params
for op in root.iter('OperationalPoint'):
	for track in op.iter('OPTrack'):
		for tunnel in track.iter('OPTrackTunnel'):
			for param in tunnel.findall('OPTrackTunnelParameter'):
				if param.attrib['IsApplicable'] != 'Y':
					# print('removed param in tunnel ', tunnel.findall('OPTrackTunnelIdentification')[0].attrib['Value'])
					tunnel.remove(param)
		for Platform in track.iter('OPTrackPlatform'):
			for param in Platform.findall('OPTrackPlatformParameter'):
				if param.attrib['IsApplicable'] != 'Y':
					# print('removed param in platform ', Platform.findall('OPTrackPlatformIdentification')[0].attrib['Value'])
					Platform.remove(param)

# BorderPoint finder
for op in root.iter('OperationalPoint'):
	id = op.findall('UniqueOPID')[0]
	# print(id)
	if id.attrib['Value'].startswith("EU"):
		print('re-wrote ', op.findall('UniqueOPID')[0].attrib['Value'], ' as OperationalBorderPoint')
		op.tag = 'OperationalBorderPoint'



list_of_smth = [
	('OperationalPoint/OPGeographicLocation',
	'OperationalBorderPoint/OPGeographicLocation'),

	('OperationalPoint',
	'OperationalBorderPoint'),

	('OperationalPoint/OPRailwayLocation',
	'OperationalBorderPoint/OPRailwayLocation'),

	('OperationalPoint/OPTrack',
	'OperationalBorderPoint/OPTrack'),

	('OperationalPoint/OPTrack/OPTrackParameter',
	'OperationalBorderPoint/OPTrack/OPTrackParameter'),

	('OperationalPoint/OPTrack/OPTrackTunnel',
	'OperationalBorderPoint/OPTrack/OPTrackTunnel'),

	('OperationalPoint/OPTrack/OPTrackTunnel/OPTrackTunnelParameter',
	'OperationalBorderPoint/OPTrack/OPTrackTunnel/OPTrackTunnelParameter'),

	('OperationalPoint/OPTrack/OPTrackPlatform',
	'OperationalBorderPoint/OPTrack/OPTrackPlatform'),

	('OperationalPoint/OPTrack/OPTrackPlatform/OPTrackPlatformParameter',
	'OperationalBorderPoint/OPTrack/OPTrackPlatform/OPTrackPlatformParameter'),
]

for smth in list_of_smth:
	iter = 0

	op = root.findall(smth[0])
	obp = root.findall(smth[1])

	ops = op + obp
	this = smth[0].split('/')[-1]
	for op in ops:
		op.attrib['num'] = this + str(iter)
		iter += 1



# tree.write('src/ES-mod.xml')
tree.write('src/ES-mini-mod-x2.xml')

## new csv conversion, after preprocessing

In [ ]:
def null_checker(object, to_find, attribute):
    temp_found = object.find(to_find)
    if temp_found is None:
        return None
    return temp_found.attrib[attribute]

In [ ]:
import pandas as pd
# import numpy as np
# import elementpath
import xml.etree.ElementTree as et


ops_cols = [
    "num", 
    "UniqueOPID", 
    "OPName", 
    "OPTafTapCode", 
    "OPType@Value", 
    "OPType@OptionalValue",
    "OPGeographicLocation@Longitude",
    "OPGeographicLocation@Latitude",
    "OPGeographicLocation@num",
    "MemberStateCode",
    "ValidityDateStart",
    "ValidityDateEnd",
]
ops_rows = []

opBs_cols = [
    "num", 
    "UniqueOPID", 
    "OPName", 
    "OPTafTapCode", 
    "OPType@Value", 
    "OPType@OptionalValue",
    "OPGeographicLocation@Longitude",
    "OPGeographicLocation@Latitude",
    "OPGeographicLocation@num",
    "MemberStateCode",
    "ValidityDateStart",
    "ValidityDateEnd",
]
opBs_rows = []

oprls_cols = [
    'num',
    'Kilometer',
    'NationalIdentNum',
    "OperationalPoint_num",
    "ValidityDateStart",
    "ValidityDateEnd",
]
oprls_rows = []

tracks_cols = [
    'num',
    'OPTrackIMCode',
    'OPTrackIdentification',
    "OperationalPoint_num",
    "ValidityDateStart",
    "ValidityDateEnd",
]
tracks_rows = []

track_param_cols = [
    'num',
    'ID',
    'Value',
    "OPTrack_num",
    "ValidityDateStart",
    "ValidityDateEnd",
]
track_param_rows = []

track_tunnels_cols = [
    'num',
    'OPTrackTunnelIMCode',
    'OPTrackTunnelIdentification',
    "OPTrack_num",
    "ValidityDateStart",
    "ValidityDateEnd",
]
track_tunnels_rows = []

track_tunnel_param_cols = [
    'num',
    'ID',
    'Value',
    "OPTrackTunnel_num",
    "ValidityDateStart",
    "ValidityDateEnd",
]
track_tunnel_param_rows = []
  
# Parsing the XML file
xmlparse = et.parse('src/ES-mini-mod-x2.xml')
# xmlparse = et.parse('src/ES-mod.xml')
root = xmlparse.getroot()

ops = root.findall('OperationalPoint')
for op in ops:

    ops_rows.append({
        "num": op.attrib['num'],
        "UniqueOPID": null_checker(op, 'UniqueOPID', 'Value'),
        "OPName": null_checker(op, 'OPName', 'Value'),
        "OPTafTapCode": null_checker(op, 'OPTafTapCode', 'Value'),
        "OPType@Value": null_checker(op, 'OPType', 'Value'),
        "OPType@OptionalValue": null_checker(op, 'OPType', 'OptionalValue'),
        "OPGeographicLocation@Longitude": null_checker(op, 'OPGeographicLocation', 'Longitude'),
        "OPGeographicLocation@Latitude": null_checker(op, 'OPGeographicLocation', 'Latitude'),
        "OPGeographicLocation@num": null_checker(op, 'OPGeographicLocation', 'num'),
        "MemberStateCode": null_checker(op, 'MemberStateCode', 'Code'),
        "ValidityDateStart": op.attrib['ValidityDateStart'],
        "ValidityDateEnd": op.attrib['ValidityDateEnd'],
    })
    
    oprls = op.findall('OPRailwayLocation')
    for oprl in oprls:
        oprls_rows.append({
            'num': oprl.attrib['num'],
            'Kilometer': oprl.attrib['Kilometer'],
            'NationalIdentNum': oprl.attrib['NationalIdentNum'],
            "OperationalPoint_num": op.attrib['num'],
            "ValidityDateStart": op.attrib['ValidityDateStart'],
            "ValidityDateEnd": op.attrib['ValidityDateEnd'],
        })

    tracks = op.findall('OPTrack')
    for track in tracks:
        tracks_rows.append({
            'num': track.attrib['num'],
            "OPTrackIMCode": null_checker(track, 'OPTrackIMCode', 'Value'),
            "OPTrackIdentification": null_checker(track, 'OPTrackIdentification', 'Value'),
            "OperationalPoint_num": op.attrib['num'],
            "ValidityDateStart": op.attrib['ValidityDateStart'],
            "ValidityDateEnd": op.attrib['ValidityDateEnd'],
        })

        track_params = track.findall('OPTrackParameter')
        for track_param in track_params:
            track_param_rows.append({
                'num': track_param.attrib['num'],
                'ID': track_param.attrib['ID'],
                'Value': track_param.attrib['Value'],
                'OPTrack_num': track.attrib['num'],
                "ValidityDateStart": op.attrib['ValidityDateStart'],
                "ValidityDateEnd": op.attrib['ValidityDateEnd'],
            })

        track_tunnels = track.findall('OPTrackTunnel')
        for track_tunnel in track_tunnels:
            track_tunnels_rows.append({
                'num': track_tunnel.attrib['num'],
                'OPTrackTunnelIMCode': null_checker(track_tunnel, 'OPTrackTunnelIMCode', 'Value'),
                'OPTrackTunnelIdentification': null_checker(track_tunnel, 'OPTrackTunnelIdentification', 'Value'),
                'OPTrack_num': track.attrib['num'],
                "ValidityDateStart": op.attrib['ValidityDateStart'],
                "ValidityDateEnd": op.attrib['ValidityDateEnd'],
            })

            track_tunnel_params = track_tunnel.findall('OPTrackTunnelParameter')
            for track_tunnel_param in track_tunnel_params:
                track_tunnel_param_rows.append({
                    'num': track_tunnel_param.attrib['num'],
                    'ID': track_tunnel_param.attrib['ID'],
                    'Value': track_tunnel_param.attrib['Value'],
                    'OPTrackTunnel_num': track_tunnel.attrib['num'],
                    "ValidityDateStart": op.attrib['ValidityDateStart'],
                    "ValidityDateEnd": op.attrib['ValidityDateEnd'],
                })



ops = root.findall('OperationalBorderPoint')
for op in ops:

    opBs_rows.append({
        "num": op.attrib['num'],
        "UniqueOPID": null_checker(op, 'UniqueOPID', 'Value'),
        "OPName": null_checker(op, 'OPName', 'Value'),
        "OPTafTapCode": null_checker(op, 'OPTafTapCode', 'Value'),
        "OPType@Value": null_checker(op, 'OPType', 'Value'),
        "OPType@OptionalValue": null_checker(op, 'OPType', 'OptionalValue'),
        "OPGeographicLocation@Longitude": null_checker(op, 'OPGeographicLocation', 'Longitude'),
        "OPGeographicLocation@Latitude": null_checker(op, 'OPGeographicLocation', 'Latitude'),
        "OPGeographicLocation@num": null_checker(op, 'OPGeographicLocation', 'num'),
        "MemberStateCode": null_checker(op, 'MemberStateCode', 'Code'),
        "ValidityDateStart": op.attrib['ValidityDateStart'],
        "ValidityDateEnd": op.attrib['ValidityDateEnd'],
    })

    oprls = op.findall('OPRailwayLocation')
    for oprl in oprls:
        oprls_rows.append({
            'num': oprl.attrib['num'],
            'Kilometer': oprl.attrib['Kilometer'],
            'NationalIdentNum': oprl.attrib['NationalIdentNum'],
            "OperationalPoint_num": op.attrib['num'],
            "ValidityDateStart": op.attrib['ValidityDateStart'],
            "ValidityDateEnd": op.attrib['ValidityDateEnd'],
        })

    tracks = op.findall('OPTrack')
    for track in tracks:
        tracks_rows.append({
            'num': track.attrib['num'],
            "OPTrackIMCode": null_checker(track, 'OPTrackIMCode', 'Value'),
            "OPTrackIdentification": null_checker(track, 'OPTrackIdentification', 'Value'),
            "OperationalPoint_num": op.attrib['num'],
            "ValidityDateStart": op.attrib['ValidityDateStart'],
            "ValidityDateEnd": op.attrib['ValidityDateEnd'],
        })
    
        track_params = track.findall('OPTrackParameter')
        for track_param in track_params:
            track_param_rows.append({
                'num': track_param.attrib['num'],
                'ID': track_param.attrib['ID'],
                'Value': track_param.attrib['Value'],
                'OPTrack_num': track.attrib['num'],
                "ValidityDateStart": op.attrib['ValidityDateStart'],
                "ValidityDateEnd": op.attrib['ValidityDateEnd'],
            })

        track_tunnels = track.findall('OPTrackTunnel')
        for track_tunnel in track_tunnels:
            track_tunnels_rows.append({
                'num': track_tunnel.attrib['num'],
                'OPTrackTunnelIMCode': null_checker(track_tunnel, 'OPTrackTunnelIMCode', 'Value'),
                'OPTrackTunnelIdentification': null_checker(track_tunnel, 'OPTrackTunnelIdentification', 'Value'),
                'OPTrack_num': track.attrib['num'],
                "ValidityDateStart": op.attrib['ValidityDateStart'],
                "ValidityDateEnd": op.attrib['ValidityDateEnd'],
            })

            track_tunnel_params = track_tunnel.findall('OPTrackTunnelParameter')
            for track_tunnel_param in track_tunnel_params:
                track_tunnel_param_rows.append({
                    'num': track_tunnel_param.attrib['num'],
                    'ID': track_tunnel_param.attrib['ID'],
                    'Value': track_tunnel_param.attrib['Value'],
                    'OPTrackTunnel_num': track_tunnel.attrib['num'],
                    "ValidityDateStart": op.attrib['ValidityDateStart'],
                    "ValidityDateEnd": op.attrib['ValidityDateEnd'],
                })




# print(opr)
  
df_ops = pd.DataFrame(ops_rows, columns=ops_cols)
df_opBs = pd.DataFrame(opBs_rows, columns=ops_cols)
df_oprls = pd.DataFrame(oprls_rows, columns=oprls_cols)
df_tracks = pd.DataFrame(tracks_rows, columns=tracks_cols)
df_track_param = pd.DataFrame(track_param_rows, columns=track_param_cols)
df_track_tunnels = pd.DataFrame(track_tunnels_rows, columns=track_tunnels_cols)
df_track_tunnel_param = pd.DataFrame(track_tunnel_param_rows, columns=track_tunnel_param_cols)


# print(df_ops)
print(df_track_tunnel_param)
  
# Writing dataframe to csv
df_ops.to_csv('src/df_ops.csv')
df_opBs.to_csv('src/df_opBs.csv')
df_oprls.to_csv('src/df_oprls.csv')
df_tracks.to_csv('src/df_tracks.csv')
df_track_param.to_csv('src/df_track_param.csv')
df_track_tunnels.to_csv('src/df_track_tunnels.csv')
df_track_tunnel_param.to_csv('src/df_track_tunnel_param.csv')